## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import h5py
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import load_model

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [ ]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()

print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

display(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# set threshold at 200, then flag any value under that amount
threshold = 200
application_types_to_replace = application_type_counts[application_type_counts < threshold].index

# replace less frequent types with "Other"
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, 'Other', inplace=True)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

display(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
display(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# set threshold at 1000, flag any value under
threshold = 1000
classifications_to_replace = classification_counts[classification_counts < threshold].index

# replace less frequent types with "Other"
application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other', inplace=True)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`

dummy_columns_list = ['AFFILIATION', 'USE_CASE', 'ORGANIZATION']

# Use pd.get_dummies() to convert the categorical column into dummy variables
data_with_dummies = pd.get_dummies(application_df[dummy_columns_list])

# Concatenate the original DataFrame with the dummy variables
# axis=1 specifies to concatenate columns
df_dummied = pd.concat([application_df, data_with_dummies], axis=1)

# Drop the original categorical column if needed
df_dummied.drop(dummy_columns_list, axis=1, inplace=True)

# explicit print - check out results
display(df_dummied)

,APPLICATION_TYPE,CLASSIFICATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,T10,C1000,1,0,N,5000,1,False,False,True,...,False,False,False,False,False,True,True,False,False,False
1,T3,C2000,1,1-9999,N,108590,1,False,False,True,...,False,False,False,False,True,False,False,True,False,False
2,T5,C3000,1,0,N,5000,0,True,False,False,...,False,False,False,False,False,True,True,False,False,False
3,T3,C2000,1,10000-24999,N,6692,1,True,False,False,...,False,False,False,False,True,False,False,False,False,True
4,T3,C1000,1,100000-499999,N,142590,1,False,False,True,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,C1000,1,0,N,5000,0,False,False,True,...,False,False,False,False,False,True,True,False,False,False
34295,T4,C3000,1,0,N,5000,0,True,False,False,...,False,False,False,False,False,True,True,False,False,False
34296,T3,C2000,1,0,N,5000,0,True,False,False,...,False,False,False,False,True,False,True,False,False,False
34297,T5,C3000,1,0,N,5000,1,False,False,True,...,False,False,False,False,False,True,True,False,False,False


In [ ]:
# added cell - remove leading 'T' from application_type and 'C' from classification so that columns becomes scalable
df_dummied['APPLICATION_TYPE'] = df_dummied['APPLICATION_TYPE'].str.replace('T', '').replace('Other', '0')

# Convert the column to integer
df_dummied['APPLICATION_TYPE'] = df_dummied['APPLICATION_TYPE'].astype(int)

df_dummied['CLASSIFICATION'] = df_dummied['CLASSIFICATION'].str.replace('C', '').replace('Other', '0')

# Convert the column to integer
df_dummied['CLASSIFICATION'] = df_dummied['CLASSIFICATION'].astype(int)


In [ ]:
# added cell - clean up income_amt column
df_dummied['INCOME_AMT'] = df_dummied['INCOME_AMT'].str.replace('M+', '000000')  # Convert '50M+' to '50000000'

# Handle specific values like '50M+', '1M-5M', '10M-50M', and '0'
df_dummied['INCOME_AMT'] = df_dummied['INCOME_AMT'].replace({'50M+': '50000000', '1M-5M': '3000000', '5M-10M': '75000000', '10M-50M': '30000000', '0': '0'})

# Convert ranges to numeric values (taking the average)
df_dummied['INCOME_AMT'] = df_dummied['INCOME_AMT'].apply(lambda x: sum(map(int, x.split('-'))) / 2 if '-' in x else int(x))

# Convert the column to integer
df_dummied['INCOME_AMT'] = df_dummied['INCOME_AMT'].astype(int)


In [ ]:
# added cell - reformat special_consideration from Y/N to 0/1
df_dummied['SPECIAL_CONSIDERATIONS'] = df_dummied['SPECIAL_CONSIDERATIONS'].map({'Y': 1, 'N': 0})


In [ ]:
# Split our preprocessed data into our features and target arrays
X = df_dummied.drop(columns=['IS_SUCCESSFUL'])
y = df_dummied['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# added cell - output csvs to troubleshoot

# Convert X_train and y_train to DataFrames
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)

# Save X_train and y_train to CSV files
X_train_df.to_csv('X_train.csv', index=False)
y_train_df.to_csv('y_train.csv', index=False)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=21, activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                220       
                                                                 
 dense_1 (Dense)             (None, 6)                 66        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 293 (1.14 KB)
Trainable params: 293 (1.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Convert data types to float32
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

# Convert X_test and y_test data types to float32
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
# Train the model
history = nn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
858/858 [==============================] - 3s 2ms/step - loss: 83956.9688 - accuracy: 0.4757 - val_loss: 19920.9121 - val_accuracy: 0.4526
Epoch 2/10
858/858 [==============================] - 2s 2ms/step - loss: 6885.4487 - accuracy: 0.4815 - val_loss: 7473.6304 - val_accuracy: 0.4706
Epoch 3/10
858/858 [==============================] - 2s 2ms/step - loss: 3663.1868 - accuracy: 0.5048 - val_loss: 4990.9736 - val_accuracy: 0.4671
Epoch 4/10
858/858 [==============================] - 2s 3ms/step - loss: 3323.1826 - accuracy: 0.5073 - val_loss: 9119.4326 - val_accuracy: 0.5335
Epoch 5/10
858/858 [==============================] - 3s 3ms/step - loss: 7743.9531 - accuracy: 0.4946 - val_loss: 4119.1465 - val_accuracy: 0.5821
Epoch 6/10
858/858 [==============================] - 2s 2ms/step - loss: 2067.3389 - accuracy: 0.5182 - val_loss: 1744.5269 - val_accuracy: 0.5889
Epoch 7/10
858/858 [==============================] - 2s 2ms/step - loss: 5115.9468 - accuracy: 0.5308 - val_l

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.8033 - accuracy: 0.6850 - 278ms/epoch - 1ms/step
Loss: 0.8033393621444702, Accuracy: 0.6849853992462158


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# added cell - in-depth model stats and confusion matrix

# Load the model from the HDF5 file
model_path = "AlphabetSoupCharity.h5"
model = load_model(model_path)

# Make predictions on the test data
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Assuming y_test and y_pred are one-dimensional arrays
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
target_names = ["Class 0", "Class 1"]  # Update class names as needed
class_report = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:")
print(class_report)

215/215 [==============================] - 0s 1ms/step
Confusion Matrix:
[[3200    0]
 [3660    0]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.47      1.00      0.64      3200
     Class 1       0.00      0.00      0.00      3660

    accuracy                           0.47      6860
   macro avg       0.23      0.50      0.32      6860
weighted avg       0.22      0.47      0.30      6860



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
